In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [3]:
day = date(2024, 3, 1)
day_dateoffset = day - pd.DateOffset(months=1)
print(day_dateoffset)

2024-02-01 00:00:00


In [4]:
day = date(2024, 3, 1)
day_timedelta = day - timedelta(days=31)
print(day_timedelta)

2024-01-30


In [5]:
f = open('call_rate_test.csv', 'r', encoding='utf-8')

# 데이터를 완전히 동일하게 가져올 수 있는가
headers = f.readline().rstrip() # 헤더 읽기, 맨 끝 줄바꿈기호 삭제
print(headers.split('\t'))
headers = headers.split('\t') # 같은 이름의 변수에 리스트로 바꾼 것 넣어주기

# 데이터 읽기
lines = f.readlines()
print(lines[:5])

new_datas = []
for line in lines: # 한 줄씩 
    data = line.rstrip().split('\t') # 줄바꿈 기호 지우고, ,로 잘라준다.
    dict_data = { header: data[i] for i, header in enumerate(headers) }
    new_datas.append(dict_data) # 데이터를 딕셔너리로 만들어서 넣어야된다.

f.close()

['date', '종가', '등락률']
['2023-10-19\t3.5050\t0.37\n', '2023-10-18\t3.4920\t-0.02\n', '2023-10-17\t3.4930\t-0.71\n', '2023-10-16\t3.5180\t-0.67\n', '2023-10-13\t3.5420\t0.85\n']


In [6]:
def find_nearest_date(df, target_date):
    delta = pd.Timedelta(days=1)
    nearest_date = None
    min_difference = pd.Timedelta.max

    for date in df['date']:
        difference = abs(target_date - date)
        if difference < min_difference:
            min_difference = difference
            nearest_date = date

    return nearest_date

def up_down_call(df):
    df_ud = pd.DataFrame(index=df.index, columns=['date', '종가', 'up_down'])

    # '날짜' 열을 datetime 형식으로 변환
    df['date'] = pd.to_datetime(df['date'])

    for i, row in df.iterrows():
        time = row['date']
        time2 = time - pd.DateOffset(months=1)

        # 이전 달의 날짜가 인덱스에 있는지 확인
        if time2 in df['date'].values:
            prev_row = df[df['date'] == time2].iloc[0]
        else:
            # 이전 날짜가 없는 경우 가장 가까운 날짜로 설정
            nearest_date = find_nearest_date(df, time2)
            prev_row = df[df['date'] == nearest_date].iloc[0]

        if row['종가'] > prev_row['종가']:
            up_down = '매파'
        elif row['종가'] == prev_row['종가']:
            up_down = 0
        else:
            up_down = '비둘기파'

        df_ud.loc[i] = [time.date(), row['종가'], up_down]

    return df_ud

In [7]:
# # 예시 데이터
# df = pd.DataFrame({
#     '날짜': ['2024-03-21', '2024-03-22', '2024-03-23'],
#     '종가': [3.5290, 3.5900, 3.5800]
# })
df = pd.DataFrame(new_datas)

result = up_down_call(df)
# result['content'] = None
result

,date,종가,up_down
0,2023-10-19,3.5050,비둘기파
1,2023-10-18,3.4920,비둘기파
2,2023-10-17,3.4930,비둘기파
3,2023-10-16,3.5180,비둘기파
4,2023-10-13,3.5420,비둘기파
5,2023-10-12,3.5120,비둘기파
6,2023-10-05,3.5270,비둘기파
7,2023-10-04,3.5550,비둘기파
8,2023-09-27,3.8290,매파
9,2023-09-26,3.6200,매파


In [8]:
new_df = pd.read_json("news_test.json")
new_df = new_df.sort_values(by=["date"], ascending=False)
# 데이터프레임 확인
# print(new_df)
new_df = pd.DataFrame(new_df)
new_df

,date,content
6,2023-10-19,원/달러 환율이 장중 한때 1360원에 근접했다. 중동분쟁 긴장감이 고조되고 미국 ...
2,2023-10-13,나흘 연속 오르던 뉴욕증시가 인플레이션 재발우려와 국채수익률 반등으로 인해 하락세로...
4,2023-10-13,나흘 연속 오르던 뉴욕증시가 인플레이션 재발우려와 국채수익률 반등으로 인해 하락세로...
7,2023-10-12,현재 미국 금융시장은 금리 하락세가 막을 내리는 거대한 변화를 겪고 있어 주식 비중...
0,2023-10-04,미국 국채 금리 급등이 국내 가계대출 금리 상승으로 이어질 것으로 전망된다. 추석 ...
1,2023-10-04,재닛 옐런 미국 재무장관이 금리가 장기간 상승세를 유지할 것이란 전망은 '기정사실'...
3,2023-09-26,"유럽중앙은행(ECB)이 최근 금리인상 종료를 시사했지만, 금리인하의 통화완화를 논하..."
5,2023-09-26,"삼성증권이 에 대한 투자의견을 '매수', 목표주가를 5900원으로 유지했다. ..."
9,2023-01-10,세계 각국 중앙은행들이 인플레이션(물가상승)을 잡기 위해 빠른 속도로 기준금리를 끌...
10,2023-01-10,새해 첫 주나 설 명절 전후 쏟아지던 은행 특판(특별판매) 예금이 자취를 감췄다. ...


In [9]:
pip install ekonlpy

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [10]:
import konlpy as kn
import MeCab as mc
import ekonlpy as ek
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [11]:
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK
mpck = MPCK()

df = pd.read_json("news_test.json")
df = df.sort_values(by=["date"], ascending=False)

df = pd.DataFrame(df)

df['token']="" #빈 값을 가진 token column 추가

for i, row in df.iterrows() :

    try :
        tmp_df = df['content']
        #임시 데이터프레임에 불러온 기사의 본문 내용 리스트로 저장

        tokens = mpck.tokenize(tmp_df[i])
        row['token'] = '.'.join(tokens)
        df.loc[i, 'token'] = row['token']
        #eKoNLPy로 tokenizing 한 것을 데이터프레임에 column 추가

        print("success")
        if i % 100 == 0 :
            print('{}/{} : 성공'.format(i, df.shape[0]))
    except Exception as ex :
        print("fail")
        #print("{} : {}".format(i, str(ex)))

# df.to_csv("news_test.csv", mode='w')

success
success
success
success
success
0/20 : 성공
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


In [12]:
import re

# 정규표현식을 사용하여 한글 단어와 쉼표를 추출하는 함수
def extract_hangul_words(text):
    words = re.findall('[가-힣]+\,', text)
    return words

# 시리즈의 각 항목에 함수를 적용하여 한글 단어와 쉼표를 추출
df['token'] = df['token'].apply(extract_hangul_words)

In [15]:
df['token']

6     [달러, 장중, 근접, 하, 분쟁, 긴장, 고조, 되, 만기, 국채, 금리, 오르,...
2     [연속, 오르, 인플레이션, 재발, 우려, 국채, 수익률, 상승, 인하, 하락, 돌...
4     [연속, 오르, 인플레이션, 재발, 우려, 국채, 수익률, 상승, 인하, 하락, 돌...
7     [금융시장, 금리, 하락, 막, 내리, 변화, 겪, 주식비중, 줄, 채권투자, 늘,...
0     [국채, 금리, 급등, 가계, 대출, 금리, 상승, 이어지, 전망, 되, 추석, 끝...
1     [금리, 장기간, 상승, 유지, 하, 전망, 기정사실, 아니, 선, 그, 경제, 부...
3     [금리, 인상, 종료, 시사, 하, 금리, 인하, 통화, 완화, 논하, 아직, 이르...
5     [투자의견, 매수, 목표가격, 유지, 하, 대출, 만기, 고금리, 지친, 시장, 부...
9     [세계, 각국, 인플레이션, 물가, 상승, 잡, 위하, 빠르, 속도, 금리, 오르,...
10    [새해, 주, 설, 명절, 전후, 쏟, 은행, 특판, 특별, 판매, 예금, 자취, ...
8     [국민, 생활, 가장, 중요, 물가, 목표, 수준, 상회, 하, 오름, 이, 예상,...
13    [채권시장, 양상, 보, 예상, 되, 금리, 인상, 멈추, 채권, 금리, 안정화, ...
11    [하락, 마감, 하, 대거, 약세, 보, 나스닥, 하락, 하, 현지, 다우존스산업평...
12    [더불, 금리, 상승, 서민, 실수요자, 대출, 어렵, 덜, 사이클, 위하, 변동,...
14    [뉴스, 기자, 신종, 코로나, 바이러스, 코로나, 여파, 금리, 포인트, 인하, ...
15    [증시, 상승, 하락, 거듭, 하, 미, 포인트, 금리, 인하, 불구, 여전히, 불...
16    [시장, 흔드, 이슈, 날카롭, 시선, 분석, 하, 장, 대응, 전략, 제시, 하,...
17    [임시, 소집, 하, 금리, 인하, 코로나, 확산, 따르, 경제, 충격, 완

In [34]:
df

,date,content,token
6,2023-10-19,원/달러 환율이 장중 한때 1360원에 근접했다. 중동분쟁 긴장감이 고조되고 미국 ...,"[달러, 장중, 근접, 하, 분쟁, 긴장, 고조, 되, 만기, 국채, 금리, 오르,..."
2,2023-10-13,나흘 연속 오르던 뉴욕증시가 인플레이션 재발우려와 국채수익률 반등으로 인해 하락세로...,"[연속, 오르, 인플레이션, 재발, 우려, 국채, 수익률, 상승, 인하, 하락, 돌..."
4,2023-10-13,나흘 연속 오르던 뉴욕증시가 인플레이션 재발우려와 국채수익률 반등으로 인해 하락세로...,"[연속, 오르, 인플레이션, 재발, 우려, 국채, 수익률, 상승, 인하, 하락, 돌..."
7,2023-10-12,현재 미국 금융시장은 금리 하락세가 막을 내리는 거대한 변화를 겪고 있어 주식 비중...,"[금융시장, 금리, 하락, 막, 내리, 변화, 겪, 주식비중, 줄, 채권투자, 늘,..."
0,2023-10-04,미국 국채 금리 급등이 국내 가계대출 금리 상승으로 이어질 것으로 전망된다. 추석 ...,"[국채, 금리, 급등, 가계, 대출, 금리, 상승, 이어지, 전망, 되, 추석, 끝..."
1,2023-10-04,재닛 옐런 미국 재무장관이 금리가 장기간 상승세를 유지할 것이란 전망은 '기정사실'...,"[금리, 장기간, 상승, 유지, 하, 전망, 기정사실, 아니, 선, 그, 경제, 부..."
3,2023-09-26,"유럽중앙은행(ECB)이 최근 금리인상 종료를 시사했지만, 금리인하의 통화완화를 논하...","[금리, 인상, 종료, 시사, 하, 금리, 인하, 통화, 완화, 논하, 아직, 이르..."
5,2023-09-26,"삼성증권이 에 대한 투자의견을 '매수', 목표주가를 5900원으로 유지했다. ...","[투자의견, 매수, 목표가격, 유지, 하, 대출, 만기, 고금리, 지친, 시장, 부..."
9,2023-01-10,세계 각국 중앙은행들이 인플레이션(물가상승)을 잡기 위해 빠른 속도로 기준금리를 끌...,"[세계, 각국, 인플레이션, 물가, 상승, 잡, 위하, 빠르, 속도, 금리, 오르,..."
10,2023-01-10,새해 첫 주나 설 명절 전후 쏟아지던 은행 특판(특별판매) 예금이 자취를 감췄다. ...,"[새해, 주, 설, 명절, 전후, 쏟, 은행, 특판, 특별, 판매, 예금, 자취, ..."


In [ ]:
# Date 열을 datetime으로 변환하고 인덱스로 설정
result['date'] = pd.to_datetime(result['date'])
result.set_index('date', inplace=True)

new_df['date'] = pd.to_datetime(new_df['date'])
new_df.set_index('date', inplace=True)

# 데이터프레임을 날짜를 기준으로 합치기
merged_df = pd.merge(result, new_df, on='date', how='outer')

# 결과 출력
print(merged_df)

                종가 up_down                                            content
date                                                                         
2023-10-19  3.5050    비둘기파  원/달러 환율이 장중 한때 1360원에 근접했다. 중동분쟁 긴장감이 고조되고 미국 ...
2023-10-18  3.4920    비둘기파                                                NaN
2023-10-17  3.4930    비둘기파                                                NaN
2023-10-16  3.5180    비둘기파                                                NaN
2023-10-13  3.5420    비둘기파  나흘 연속 오르던 뉴욕증시가 인플레이션 재발우려와 국채수익률 반등으로 인해 하락세로...
2023-10-13  3.5420    비둘기파  나흘 연속 오르던 뉴욕증시가 인플레이션 재발우려와 국채수익률 반등으로 인해 하락세로...
2023-10-12  3.5120    비둘기파  현재 미국 금융시장은 금리 하락세가 막을 내리는 거대한 변화를 겪고 있어 주식 비중...
2023-10-05  3.5270    비둘기파                                                NaN
2023-10-04  3.5550    비둘기파  미국 국채 금리 급등이 국내 가계대출 금리 상승으로 이어질 것으로 전망된다. 추석 ...
2023-10-04  3.5550    비둘기파  재닛 옐런 미국 재무장관이 금리가 장기간 상승세를 유지할 것이란 전망은 '기정사실'...
2023-09-27  3.8290      매파                                      

In [12]:
train_list = merged_df['content'].tolist()
train_list

['원/달러 환율이 장중 한때 1360원에 근접했다. 중동분쟁 긴장감이 고조되고 미국 10년 만기 국채 금리가 4.9%까지 치솟는 등 시장에 위험자산 회피 심리가 강화하는 분위기다.  19일 서울 외환시장에서 원/달러 환율은 전날(1349.6원)보다 7.8원 오른 1357.4원에 마감했다.  이날 원/달러 환율은 전 거래일보다 6.4원 오른 1356원에 거래를 시작했다. 이후 상승폭을 키워 장중 한때 1359.1원까지 오르며 1360원에 육박했다.  원/달러 환율 상승은 간밤 미국 국채금리 상승 영향으로 분석된다.  간밤 글로벌 채권 금리 벤치마크 역할을 하는 미 국채 10년물 수익률은 장중 4.93%까지 올랐다. 10년물 금리가 4.9%대를 돌파한 건 2007년 이후 약 16년 만에 처음이다.  미 국채 금리가 오른 건 미국 소비 호조로 연방준비제도(Fed, 연준)의 추가 긴축 우려가 커진 영향이다. 미국 상무부 발표에 따르면 미국의 9월 소매 판매는 7049억달러(약 955조원)로 전월 대비 0.7% 증가했다. 시장 전망치(0.2%)를 크게 웃돈 수준이다.  여기에 미국이 우크라이나에 이어 이스라엘에도 지원 약속을 하며 국채 발행이 늘어날 것이란 전망이 나오는 것도 국채 금리를 끌어올리고 있다.  이에 시장에 안전자산 선호 심리가 확대됐고 미 국채 금리가 급등했다.  대표적 안전자산인 달러화도 강세로 돌아섰다. 주요 6개국 통화 대비 달러화 가치를 나타내는 달러인덱스는 이날 한때 106.66까지 올랐다.  이스라엘과 팔레스타인 무장정파 하마스 간 분쟁도 원/달러 환율을 끌어올린 요인으로 분석된다.  미국 조 바이든 미국 대통령이 이스라엘을 방문했지만 성과는 기대보다 미흡했단 평가다. 당초 예정돼 있던 이집트·요르단·팔레스타인 지도자들과 4자 회담이 취소되면서 단기간 내 중동 안정이 어려울 수 있다는 우려가 증폭되면서다.  이에 국제유가는 상승했다. 18일(현지시간) 미국 뉴욕상업거래소에서 서부텍사스산원유(WTI) 가격은 전거래일 대비 1.92% 오른 배럴당

In [15]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

In [16]:
import re 

X_train = str(train_list)

# 2. 특성 추출 (형태소 분석 및 N-그램)
vocab = [token.split() for token in X_train]

# 3. N-그램 추출
phrases = Phrases([x for x in vocab], min_count=1, threshold=1.3)
ngram_X_train = [' '.join(phrases[x]) for x in vocab]
New_test_ngrams = ngram_X_train

# 4. Ngram2Vec 모델 학습
model = Word2Vec([x.split() for x in New_test_ngrams], vector_size=300, window=5, min_count=15)

In [17]:
model

In [9]:
# def preprocess_and_tokenize(text):

#     tokens = [token for token in mecab.pos(text) if token[1] in ['NNG','NNP', 'VV', 'VA', 'VX']]
    
#     return tokens
# # "date"를 기준으로 그룹화하여 각 그룹에 대해 토큰화 작업 수행
# grouped_tokens = new_df.groupby("date")["content"].apply(lambda x: [preprocess_and_tokenize(news) for news in x])
# grouped_tokens

date
2020-03-16    [[(서울, NNP), (뉴스, NNG), (김서연, NNP), (기자, NNG),...
2022-01-13    [[(더불, VV), (민주당, NNP), (금리, NNG), (상승, NNG), ...
2022-01-14    [[(뉴욕, NNP), (증시, NNG), (하락, NNG), (마감, NNG), ...
2023-01-01    [[(이창용, NNP), (한국은행, NNP), (총재, NNG), (국민, NNG...
2023-01-10    [[(세계, NNG), (각국, NNG), (중앙은행, NNG), (인플레이션, N...
2023-09-21    [[(미국, NNP), (연방, NNG), (준비, NNG), (위원회, NNG),...
2023-09-26    [[(유럽, NNP), (중앙은행, NNG), (최근, NNG), (금리, NNG)...
2023-10-04    [[(재닛, NNP), (옐런, NNP), (미국, NNP), (재무, NNG), ...
2023-10-12    [[(미국, NNP), (금융, NNG), (시장, NNG), (금리, NNG), ...
2023-10-13    [[(나흘, NNG), (연속, NNG), (오르, VV), (뉴욕, NNP), (...
2023-10-19    [[(달러, NNG), (환율, NNG), (장중, NNG), (한때, NNG), ...
Name: content, dtype: object

In [11]:
csv_file = 'test6.csv'
final_df = pd.DataFrame(result)
final_df.to_csv(csv_file, sep='\t', index=False)

print("CSV 파일이 생성되었습니다:", csv_file)

CSV 파일이 생성되었습니다: test6.csv
